In [86]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [87]:
elev = pd.read_csv("measures_elev.csv", index_col=0)
area = pd.read_csv("measures_area.csv", index_col=0)
grid = pd.read_csv("measures_grid.csv", index_col=0)
frac = pd.read_csv("measures_frac.csv", index_col=0)
dens = pd.read_csv("measures_dens.csv").drop(['system:index', '.geo'], axis=1)
indx = pd.read_csv("measures_indx.csv").drop(['system:index', '.geo'], axis=1)

In [88]:
measures = elev.merge(area, left_on='city', right_on='city', how='inner')

for i in [grid, frac, dens, indx]:
    measures = measures.merge(i, left_on='city', right_on='city', how='inner')

measures.head()

,city,gradient_iqr,gradient_max,gradient_min,elevation_iqr,elevation_max,elevation_min,area_min,area_max,area_mean,...,fractal_streets,fractal_buildings,day_mean,day_stdDev,night_mean,night_stdDev,ndvi_mean,ndvi_stdDev,ndwi_mean,ndwi_stdDev
0,New York City,0.0220,0.261,0.0,5.549,17.362,-2.172,0.000000,29805.231165,940.735693,...,1.675910,1.623507,766.850760,1305.399601,151.310293,213.293338,0.188684,0.184467,0.326142,0.172623
1,Los Angeles,0.0220,0.861,0.0,14.297,125.212,77.119,0.000000,55099.213429,2074.662064,...,1.657529,1.578059,383.010164,900.355617,45.812781,103.845602,0.151579,0.143746,0.144419,0.100925
2,Chicago,0.0200,0.632,0.0,2.231,193.627,175.077,0.000000,67792.071598,2694.566730,...,1.650953,1.561960,1064.810165,1338.282236,57.818916,221.753870,0.155867,0.152960,0.538388,0.072540
3,Houston,0.0090,0.450,0.0,2.024,16.875,-0.121,0.000000,87922.170451,1442.117795,...,1.645951,1.514563,323.573247,609.189522,18.264499,53.723820,0.307336,0.188677,0.234601,0.093729
4,Phoenix,0.0035,0.031,0.0,2.092,318.511,310.681,96.625412,208608.366002,24355.289146,...,1.409317,1.184342,15.587955,37.057716,4.651462,14.016248,0.222122,0.201458,0.175803,0.119662


In [89]:
matrix = measures.drop(['city', 'gradient_min', 'gradient_max', 
                        'ndvi_stdDev', 'ndwi_stdDev', 
                        'elevation_min', 'elevation_max', 
                        'area_min', 'area_max'], axis=1).corr().reset_index().melt(id_vars='index', 
                                                                                   var_name='variable', 
                                                                                   value_name='value')

In [98]:
measures.columns

Index(['city', 'gradient_iqr', 'gradient_max', 'gradient_min', 'elevation_iqr',
       'elevation_max', 'elevation_min', 'area_min', 'area_max', 'area_mean',
       'osmid_count', 'k_avg', 'n', 'm', 'prop_4way', 'prop_deadend',
       'circuity_avg', 'length_median', 'length_mean', 'length_entropy',
       'length_entropy_log', 'orientation_entropy', 'orientation_order',
       'fractal_streets', 'fractal_buildings', 'day_mean', 'day_stdDev',
       'night_mean', 'night_stdDev', 'ndvi_mean', 'ndvi_stdDev', 'ndwi_mean',
       'ndwi_stdDev'],
      dtype='object')

In [90]:
import hvplot.pandas
import hvplot

In [95]:
# Make the heatmap with hvplot
heatmap = matrix.hvplot.heatmap(
    x="index",
    y="variable",
    C="value", 
    reduce_function=np.mean,
    frame_height=600,
    frame_width=600,
    flip_yaxis=True,
    rot=90,
    colorbar=False,
    cmap="bmw",
    xlabel="",
    ylabel="",
)

heatmap = heatmap.redim(index="reference", variable="comparison")
heatmap = heatmap.opts(toolbar=None, colorbar=True, clim=(-1, 1))

In [96]:
heatmap

:HeatMap   [reference,comparison]   (value)

In [116]:
indexing = measures[['fractal_streets', 'fractal_buildings', 
                     'ndvi_mean', 'ndwi_mean',
                     'day_mean', 'night_mean', 'prop_4way', 'circuity_avg', 'orientation_entropy',
                     'gradient_iqr',
                     'area_mean', 'length_mean']].dropna()

In [117]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [118]:
scaler = StandardScaler()
scaled = scaler.fit_transform(indexing)

In [119]:
kmeans = KMeans(n_clusters=5)
kmeans.fit(scaled)
           
indexing['label'] = kmeans.labels_